In [1]:
import numpy as np
import control as ctrl

from OrderUncertainties import *
from Benchmarks import sys_variables


In [2]:
def get_sensitivity(A, B):
    mat = OrdUnc(A)
    if mat.determineCase() == 1:
        unc = mat.distinctPos(B)
    else:
        unc = mat.multSig(B)
    return unc

def all_sensitivity(bench, K, openloop=False):
    # A_bar_i = A + BKE_i where E_i = I(n) with the (i, i) entry swapped to lambda
    if openloop:
        A = bench.A
        B = bench.A
    else:
        A = bench.A + bench.B * K
        B = bench.B * K

    sensitivity = []
    for i in range(bench.nx):
        Bi = np.zeros((bench.nx, bench.nx))
        Bi[:, i] = B[:, i]
        sensitivity.append(get_sensitivity(A, Bi))

    return sensitivity


In [3]:
res = {}
for (name, bench) in sys_variables.items():
    K = ctrl.lqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
    res[name] = list(map(lambda x: float(f"{x:.2g}"), all_sensitivity(bench, K)))
res

{'RC': [-0.059, 0.24],
 'F1': [8.5, -2.4],
 'DC': [0.00014, -0.00092],
 'CS': [54.0, 12.0, 16.0, 0.4],
 'EW': [9700.0, -3.4e-05],
 'C1': [-0.00099],
 'CC': [0.066, 0.14, -0.19],
 'D5': [0.0, 0.0, 0.0, 0.0, 0.0]}

In [5]:
sys_variables['D5'].A

array([[ 1.10799347, -1.57128874,  0.        , -0.        , -0.        ],
       [ 1.76835237,  1.09396411,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.78107667, -3.40228385, -0.        ],
       [-0.        ,  0.        ,  0.39233279,  0.56672104,  0.        ],
       [-0.        , -0.        , -0.        ,  0.        , -1.64437194]])

In [4]:
list(map(lambda x: float(f"{x:.2g}"), all_sensitivity(sys_variables['D5'], [], True)))

[0.2, 0.34, 0.0, 0.0, 0.38]